In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import itertools

In [2]:
def split(str):
    new_str = []
    str = str.split(' (')
    
    if(len(str) == 2):
        new_str.append(str[0])
        str2 = str[1].split(')')
        new_str.append(str2[0])
        str3 = str2[1].split(', ')
        if(len(str3) != 2):
            new_str.append(str3[0]+', '+str3[1])
            new_str.append(str3[2])
        else:
            for i in str3:
                new_str.append(i)
    else:
        new_str.append(str[0]+' ('+str[1])
        str2 = str[len(str)-1].split(')')
        new_str.append(str2[0])
        
        str3 = str2[1].split(', ')
        for i in str3:
            new_str.append(i)
    
    return(new_str)

In [3]:
def combine_list(list1, list2):
    list1 = list1 + list2
    del list1[-1]
    idx = 1
    len_list2 = len(list2)
    for i in range(len_list2):
        list1[len(list1)-i-idx] = list1[len(list1)-len(list2)-i]
    for i in range(len_list2):
        list1[idx+i] = list2[i]
    
    return (list1)   

In [4]:
def get_data(url):
    html = urlopen(url)
    soup = BeautifulSoup(html, "lxml")

    #set it yourself
    column_headers = ['Rank', 'Game', 'Platform', 'Publisher', 'Genre', 'Week', 'Yearly', 'Total']
    
    start_row = 4
    end_row = start_row + 100 * 2
    data_rows = soup.findAll('tr')[start_row:end_row]
    #print(data_rows)
    company_data = [[td.getText() for td in data_rows[i].findAll('td')] for i in range(len(data_rows))]
    new_company_data = [] 
    for i in range(len(data_rows)):
        company_row = [] 
        for td in data_rows[i].findAll('td'):         
            tdtext = []
            for i in td.getText().split('\n'):
                if i=="\n" or i=="\t" or i=='':
                    continue
                else:
                    tdtext.append(i)
            for j in tdtext:
                company_row.append(j)

        if(len(company_row) > 1):
            del company_row[2]        
            new_company_data.append(company_row)
            #print(new_company_data)
    
    final_company_data = []
    for change in new_company_data: 
        if(len(change) != 5):
            continue
        else:
            split_list = split(change[1])
            final_company_data.append(combine_list(change, split_list))
    
    
    df = pd.DataFrame(final_company_data, columns=column_headers)
    return (df)


In [5]:
#output excel
writer = pd.ExcelWriter('Europe Yearly Chart.xlsx')
for i in range(12, 0, -1):
    year = i + 5
    if(year < 10):
        yr = '0'+str(year)
    else:
        yr = str(year)
        
    url = 'http://www.vgchartz.com/yearly/20'+yr+'/Europe/'
    df = get_data(url)
    df.to_excel(writer,'20'+yr)
writer.save()    